In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [2]:
import os
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize']=8,5

In [53]:
#import data
os.chdir("/home/gladkov/storage/al_R/3new")
os.listdir()
otus = pd.read_csv("otu_table_for_rf_merged.txt", sep="\t")
metadata = pd.read_csv("metadata_for_rf_merged.txt", sep="\t")



In [4]:
# some function for more suitable rownames
def rownames_creator(x):
    first_column = x[0]
    repeats_column = x[2]
    repeat_number = first_column.split(".")[1]
    rownames = repeats_column + "." + repeat_number
    return rownames
# return first_column
metadata["rownames"] = metadata.apply(rownames_creator, axis=1, result_type="expand")
metadata = metadata.rename({metadata.columns[0]: "rowname_old"}, axis="columns")
metadata = metadata.set_index("rownames")
meta_col = list(metadata.columns)

# repair otu table & create nicy rownames like above
# otus.columns = otus.iloc[0]
# otus = otus.drop(["Unnamed: 0"])
# not any need to repair?
# otus.index = meta_col
meta_index = list(metadata.index)
otus.index = meta_index
otus = otus.drop(columns=["Unnamed: 0"])

IndexError: ('list index out of range', 'occurred at index 0')

In [54]:
# same shit that above with merged repeats
metadata.index = metadata["Unnamed: 0"]
metadata = metadata.drop(columns=["Unnamed: 0"])

otus.index = otus["Unnamed: 0"]
otus = otus.drop(columns=["Unnamed: 0"])

metadata = metadata.drop(columns=["Filename","Description","Plant","Inoculation","Al", "QRTPCR", "SeedCa"])

In [6]:
#create df with factor that consist only numerical data
met_col_list = list(metadata.columns)
count_list = list(range(0,105))
# np.column_stack((count_list, met_col_list))
# metadata = metadata.drop([])


In [7]:
metadata

,BineM,SeedM,BineSeedM,SeedN,SeedMOne,NodPerPlant,NodPerPlant.1,Mic,Ves,Arb,...,BineSeedPerPlantCu,BineSeedPerPlantFe,BineSeedPerPlantK,BineSeedPerPlantMg,BineSeedPerPlantMo,BineSeedPerPlantNi,BineSeedPerPlantP,BineSeedPerPlantS,BineSeedPerPlantS.1,BineSeedPerPlantZn
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
s1903,5.217500,4.342500,9.1975,40.5,121.156977,37.5,11.30,52.0000,19.35,32.00,...,205.559426,766.378262,714.277514,62.535185,76.387623,147.119788,80.156457,189.861096,47.650990,162.618635
s1903_Al,4.392500,3.815000,8.3575,32.5,116.472538,8.0,3.00,3.5375,3.50,2.65,...,180.729942,666.361386,693.159352,55.043641,73.033851,296.895381,20.194510,163.559044,40.544464,152.699049
s1903_In,5.927500,5.102500,11.0300,38.5,120.505128,75.0,22.00,41.0000,19.35,17.65,...,241.547593,1020.152172,756.427953,81.277633,90.928284,186.786847,113.749773,218.738423,59.010729,317.804489
s1903_In_Al,6.280000,4.510000,10.7600,38.0,132.236842,107.0,25.00,2.3000,6.00,4.00,...,214.902651,882.726277,1024.193155,85.113614,102.978868,432.093049,31.304417,204.789880,69.358599,197.320157
s7307,6.171667,9.320833,15.7325,37.0,245.901568,76.0,40.55,27.6500,15.00,10.00,...,349.903165,1228.812499,610.736871,79.297033,118.448587,197.922382,54.625197,346.483515,50.242766,314.062153
s7307_Al,5.790000,6.465000,12.2875,29.5,222.635417,89.0,25.90,44.6500,13.30,36.35,...,328.323829,1348.291963,814.213264,82.019465,193.789526,511.311272,30.422035,316.907947,56.710160,514.203436
s7307_In,7.550000,9.617500,17.2850,36.0,267.682228,109.0,50.00,37.1500,15.65,21.30,...,425.149237,1716.719807,964.578965,93.713259,165.309027,283.260445,84.401096,422.719562,70.984900,450.168242
s7307_In_Al,6.807500,6.432500,13.3475,28.5,229.732143,211.0,37.85,64.1500,42.65,41.30,...,309.794086,1067.851311,831.875468,92.289192,179.595986,565.965374,30.928963,298.532203,62.610131,426.562850
s8353,3.290000,4.607500,7.8975,19.5,248.727941,26.5,10.90,50.8500,17.85,44.00,...,217.580950,642.869897,450.987282,45.250730,112.997601,219.508415,96.767008,264.529665,50.103618,203.462065


In [8]:
otus

,Seq9032,Seq9292,Seq9026,Seq9514,Seq6744,Seq9467,Seq9390,Seq9750,Seq4443,Seq8153,...,Seq4725,Seq2306,Seq7279,Seq8112,Seq5958,Seq5255,Seq8827,Seq8848,Seq9482,Seq8604
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
s1903,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,13.019559,12.381450,...,11.710072,13.380344,11.710072,12.906451,11.710072,11.710072,11.710072,11.710072,11.710072,12.885218
s1903_Al,11.710072,11.710072,11.710072,11.710072,12.657625,11.710072,11.710072,11.710072,11.710072,11.710072,...,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,13.028021,11.710072,11.710072,11.710072
s1903_In,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,13.167133,12.654322,...,12.765952,13.618598,11.710072,12.246774,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072
s1903_In_Al,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,...,9.706624,9.322083,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554
s7307,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,12.177178,...,11.710072,13.543454,11.710072,11.710072,11.710072,11.710072,12.237664,11.710072,11.710072,11.710072
s7307_Al,11.710072,11.710072,11.710072,11.710072,11.710072,12.284653,11.710072,11.710072,11.710072,12.739978,...,11.710072,14.804255,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072
s7307_In,11.710072,11.710072,11.710072,11.710072,12.914762,11.710072,11.710072,11.710072,11.710072,11.710072,...,14.324443,13.400823,13.829970,12.212878,12.431779,11.710072,11.710072,12.709640,12.930595,12.701177
s7307_In_Al,11.710072,11.710072,12.564065,11.710072,11.710072,11.710072,11.710072,11.710072,13.926717,12.207882,...,13.376533,12.780527,11.710072,13.376397,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072
s8353,11.710072,12.918492,11.710072,11.710072,11.710072,12.427628,11.710072,12.471904,12.929170,12.231530,...,11.710072,15.326957,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072


In [56]:

nas = metadata.isna().any()
nas.sort_values(ascending=True)


BineM                  False
BinePerPLantZn         False
BinePerPlantSe         False
BinePerPlantP          False
BinePerPlantNi         False
                       ...  
BineMn                 False
BineMg                 False
BineK                  False
BineSeedPerPlantS.1    False
BineSeedPerPlantZn     False
Length: 104, dtype: bool

In [55]:
# Use numpy to convert to arrays
import numpy as np# Labels are the values we want to predict

feature_list = list(metadata)# Convert to numpy array
features = np.array(metadata)
labels = np.array(otus)

In [57]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [58]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (12, 104)
Training Labels Shape: (12, 9971)
Testing Features Shape: (4, 104)
Testing Labels Shape: (4, 9971)


In [111]:
# Import the model we are using

from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs = -1)# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [119]:
# Get numerical feature importances
importances = list(rf.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: BinePerPLantZn       Importance: 0.036
Variable: NitSeed              Importance: 0.028
Variable: SoilNi               Importance: 0.028
Variable: BinePerPlantFe       Importance: 0.027
Variable: BinePerPlantMg       Importance: 0.026
Variable: BinePerPLantCa       Importance: 0.023
Variable: BinePerPlantB        Importance: 0.022
Variable: BinePerPlantK        Importance: 0.022
Variable: SoilCo               Importance: 0.02
Variable: BinePerPlantP        Importance: 0.02
Variable: BineSeedPerPlantK    Importance: 0.019
Variable: SeedMo               Importance: 0.018
Variable: SeedNi               Importance: 0.018
Variable: BineSeedPerPlantCo   Importance: 0.018
Variable: NitPerPlant          Importance: 0.017
Variable: BinePerPlantCo       Importance: 0.017
Variable: BinePerPlantMn       Importance: 0.017
Variable: BinePerPlantNi       Importance: 0.017
Variable: SoilS                Importance: 0.016
Variable: BinePerPlantSe       Importance: 0.016
Variable: Arb         

In [64]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)# Calculate the absolute errors
errors = abs(predictions - test_labels)# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 1.28 degrees.


In [77]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(train_features, train_labels)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
/home/gladkov/.conda/envs/pandas/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [92]:
best_random = rf_random.best_estimator_
rf_random.best_estimator_

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=90, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [96]:
rf_2 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=90, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
rf_2.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=90, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [100]:
# Use the forest's predict method on the test data
predictions = rf_2.predict(test_features)# Calculate the absolute errors
errors = abs(predictions - test_labels)# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 1.28 degrees.


array([[11.84336729, 11.74909971, 11.66628797, ..., 11.73591848,
        11.73400929, 11.72944279],
       [11.46718302, 11.41290214, 11.55004452, ..., 11.47253112,
        11.39872961, 11.44288857],
       [11.66690231, 11.59625925, 11.77870534, ..., 11.68913733,
        11.57979746, 11.62388205],
       [11.85740913, 11.74721857, 11.82157107, ..., 11.7647791 ,
        11.69772932, 11.75237937]])

In [98]:
# Get numerical feature importances
importances = list(rf_2.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: BinePerPLantZn       Importance: 0.04
Variable: NitPerPlant          Importance: 0.03
Variable: BineMn               Importance: 0.03
Variable: SeedMo               Importance: 0.03
Variable: SoilCo               Importance: 0.03
Variable: BineSeedPerPlantCo   Importance: 0.03
Variable: BineSeedPerPlantK    Importance: 0.03
Variable: SeedN                Importance: 0.02
Variable: pH                   Importance: 0.02
Variable: NitSeed              Importance: 0.02
Variable: BineZn               Importance: 0.02
Variable: SeedMg               Importance: 0.02
Variable: SeedNi               Importance: 0.02
Variable: SoilFe               Importance: 0.02
Variable: SoilMn               Importance: 0.02
Variable: BinePerPlantB        Importance: 0.02
Variable: BinePerPlantCo       Importance: 0.02
Variable: BinePerPlantCu       Importance: 0.02
Variable: BinePerPlantFe       Importance: 0.02
Variable: BinePerPlantK        Importance: 0.02
Variable: BinePerPlantMg       Importanc

In [101]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random2 = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random2.fit(train_features, train_labels)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   38.1s
/home/gladkov/.conda/envs/pandas/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [102]:
rf_random2.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': True}

In [103]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': False}

In [104]:
rf_3 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=80, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
rf_3.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=80, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [105]:
# Use the forest's predict method on the test data
predictions = rf_3.predict(test_features)# Calculate the absolute errors
errors = abs(predictions - test_labels)# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 1.26 degrees.


In [118]:
# Get numerical feature importances
importances = list(rf_3.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: BinePerPLantCa       Importance: 0.034
Variable: BinePerPLantZn       Importance: 0.028
Variable: Mic                  Importance: 0.027
Variable: SeedMg               Importance: 0.025
Variable: SeedNi               Importance: 0.023
Variable: BinePerPlantMg       Importance: 0.022
Variable: BinePerPlantB        Importance: 0.021
Variable: Arb                  Importance: 0.02
Variable: SeedMo               Importance: 0.02
Variable: Ves                  Importance: 0.019
Variable: BinePerPlantCo       Importance: 0.019
Variable: BinePerPlantFe       Importance: 0.019
Variable: BinePerPlantNi       Importance: 0.019
Variable: BineSeedPerPlantK    Importance: 0.019
Variable: NitSeed              Importance: 0.018
Variable: SoilCo               Importance: 0.018
Variable: BineSeedPerPlantCo   Importance: 0.018
Variable: BinePerPlantAl       Importance: 0.017
Variable: NitPerPlant          Importance: 0.016
Variable: SoilNi               Importance: 0.016
Variable: BinePerPlant

In [108]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

random_accuracy = evaluate(rf_3, test_features, test_labels)

Model Performance
Average Error: 1.2571 degrees.
Accuracy = 88.72%.


In [109]:
random_accuracy = evaluate(rf_2, test_features, test_labels)

Model Performance
Average Error: 1.2812 degrees.
Accuracy = 88.50%.


In [112]:
random_accuracy = evaluate(rf, test_features, test_labels)

Model Performance
Average Error: 1.2841 degrees.
Accuracy = 88.46%.


In [59]:
BineMsumByAl = metadata \
.groupby(["Plant", "Al"]) \
.agg({"BineM":"sum", "Plant": "count"}) \
.rename(columns={"BineM":"SumBineM", "Plant":"PlantCount"}) \
.sort_values("SumBineM", ascending=False)
    

In [59]:
BineMsumByAl = metadata \
.groupby(["Plant", "Al"]) \
.agg({"BineM":"sum", "Plant": "count"}) \
.rename(columns={"BineM":"SumBineM", "Plant":"PlantCount"}) \
.sort_values("SumBineM", ascending=False)
    

In [39]:
if int(BineMsumByAl.SumBineM.sum()) == int(metadata.BineM.sum()):
    print("OK")
else:
    print("ERRoR")

OK


In [20]:
otus.dtypes

Seq9032    float64
Seq9292    float64
Seq9026    float64
Seq9514    float64
Seq6744    float64
            ...   
Seq5255    float64
Seq8827    float64
Seq8848    float64
Seq9482    float64
Seq8604    float64
Length: 9971, dtype: object

In [19]:
metadata.dtypes

BineM                  float64
SeedM                  float64
BineSeedM              float64
SeedN                  float64
SeedMOne               float64
                        ...   
BineSeedPerPlantNi     float64
BineSeedPerPlantP      float64
BineSeedPerPlantS      float64
BineSeedPerPlantS.1    float64
BineSeedPerPlantZn     float64
Length: 105, dtype: object